## Only upsampling Test data 

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
import seaborn as sns
plt.rcParams['font.size'] = 14
plt.style.use('seaborn-whitegrid')

In [2]:
df = pd.read_csv('cleaned_data/Cleaned Data.csv',index_col=[0])

In [3]:
df['Signal'].value_counts()

Sell    6153
Buy     4681
Hold     745
Name: Signal, dtype: int64

In [4]:
SignalFac = []

for string in df['Signal']:
    if string == 'Sell':
        SignalFac.append(2)
    elif string == 'Hold':
        SignalFac.append(1)    
    else:
        SignalFac.append(0)

In [5]:
df['SignalFac'] = SignalFac

In [6]:
y = df['SignalFac']
X = df.drop(['Signal','SignalFac'], axis=1)

In [8]:
# Create Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [10]:
print('Sell hat', X_train[y_train==2].shape[0], 'Werte')
print('Hold hat', X_train[y_train==1].shape[0], 'Werte')
print('Buy hat', X_train[y_train==0].shape[0], 'Werte')

Sell hat 4307 Werte
Hold hat 521 Werte
Buy hat 3277 Werte


In [11]:
#Upsampling of Hold to the number of Sell

from sklearn.utils import resample

X_upsampled, y_upsampled = resample(X_train[y_train==1], y_train[y_train==1],
                                    replace=True,
                                    n_samples=3786,
                                    random_state=1)

In [12]:
# Combine datasets
X_bal = np.vstack((X_train, X_upsampled))
y_bal = np.hstack((y_train, y_upsampled))

In [13]:
#Upsampling of Buy to the number of Sells

from sklearn.utils import resample

X2_upsampled, y2_upsampled = resample(X_train[y_train==0], y_train[y_train==0],
                                    replace=True,
                                    n_samples=1030,
                                    random_state=1)

In [14]:
# Combine datasets
X_bal = np.vstack((X_bal, X2_upsampled))
y_bal = np.hstack((y_bal, y2_upsampled))

In [15]:
pd.value_counts(pd.DataFrame(y_bal)[0])

2    4307
1    4307
0    4307
Name: 0, dtype: int64

In [16]:
from sklearn.ensemble import RandomForestClassifier

# Create classifier object and fit it to data
forest = RandomForestClassifier(criterion='gini', random_state=0, n_jobs=-1)
forest.fit(X_bal, y_bal)

RandomForestClassifier(n_jobs=-1, random_state=0)

In [18]:
y_pred = forest.predict(X_test)

In [21]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.49      0.47      0.48      1404
           1       0.06      0.01      0.02       224
           2       0.59      0.67      0.63      1846

    accuracy                           0.55      3474
   macro avg       0.38      0.38      0.38      3474
weighted avg       0.52      0.55      0.53      3474



In [23]:
print('Confusion matrix: \n', 
      metrics.confusion_matrix(y_test,y_pred))

Confusion matrix: 
 [[ 661   17  726]
 [  87    2  135]
 [ 594   12 1240]]
